# Treinamento de Modelo YOLO Customizado

Este notebook implementa o passo a passo para treinar um detector de objetos YOLOv4-tiny customizado no Google Colab.

**Pré-requisitos:**
Antes de executar este notebook, você deve ter completado a **Fase A (Preparação dos Dados)**:
1. Coletado suas imagens.
2. Rotulado com o LabelMe (gerando arquivos `.xml`).
3. Convertido as anotações para o formato YOLO (gerando arquivos `.txt`).

### Passo 1: Montar o Google Drive e Verificar a GPU

Primeiro, conectamos o Colab ao nosso Google Drive para que possamos acessar nossos arquivos. Em seguida, garantimos que uma GPU foi alocada para acelerar o treinamento.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Verifique se a GPU está ativa. Você deve ver uma tabela com informações da sua GPU NVIDIA.
# Se der erro, vá em "Ambiente de execução" -> "Alterar tipo de ambiente de execução" e selecione GPU.
!nvidia-smi

### Passo 2: Clonar e Compilar o Repositório Darknet

Baixamos o framework Darknet (versão de AlexeyAB) e o compilamos com suporte para GPU, CUDNN e OpenCV para obter o máximo de desempenho.

In [ ]:
# Clona o repositório do Darknet
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet

# Modifica o Makefile para habilitar GPU, CUDNN e OpenCV
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile # Compila a biblioteca para uso posterior em Python

# Compila o Darknet (pode levar alguns minutos)
!make

### Passo 3: Preparar a Estrutura de Arquivos (AÇÃO MANUAL)

**Esta é a etapa mais importante!** Antes de prosseguir, você deve ter a seguinte estrutura no seu Google Drive:

```
/content/drive/MyDrive/
└── yolo_treinamento/                # Pasta principal do projeto
    ├── dataset/                     # Pasta com seus dados
    │   ├── imagem01.jpg
    │   ├── imagem01.txt
    │   ├── imagem02.jpg
    │   ├── imagem02.txt
    │   └── ...
    ├── backup/                      # O Darknet salvará os pesos aqui (crie esta pasta vazia)
    ├── obj.names                    # Arquivo com os nomes das suas classes
    ├── obj.data                     # Arquivo de configuração de dados
    └── yolov4-tiny-obj.cfg          # Arquivo de configuração da rede
```

Certifique-se de que os caminhos dentro dos arquivos `obj.data` e `.cfg` estão corretos, como explicado no guia anterior.

### Passo 4: Gerar os Arquivos `train.txt` e `valid.txt`

Este código Python irá ler o conteúdo da sua pasta `dataset` e criar duas listas: uma para as imagens de treino e outra para as de validação, salvando os caminhos absolutos nos arquivos `train.txt` e `valid.txt`.

In [ ]:
import os
import random

# ATENÇÃO: Verifique se este caminho corresponde à sua estrutura no Google Drive
caminho_base_drive = '/content/drive/MyDrive/yolo_treinamento/'
caminho_dataset = os.path.join(caminho_base_drive, 'dataset/')

# Lista todas as imagens .jpg na pasta do dataset
all_images = [os.path.join(caminho_dataset, f) for f in os.listdir(caminho_dataset) if f.endswith('.jpg')]
random.shuffle(all_images)

# Divide em 80% para treino e 20% para validação
split_index = int(0.8 * len(all_images))
train_images = all_images[:split_index]
valid_images = all_images[split_index:]

# Salva os caminhos nos arquivos .txt na pasta principal do projeto
with open(os.path.join(caminho_base_drive, 'train.txt'), 'w') as f:
    f.write('\n'.join(train_images))

with open(os.path.join(caminho_base_drive, 'valid.txt'), 'w') as f:
    f.write('\n'.join(valid_images))

print(f"Arquivo 'train.txt' com {len(train_images)} imagens criado com sucesso!")
print(f"Arquivo 'valid.txt' com {len(valid_images)} imagens criado com sucesso!")

### Passo 5: Baixar Pesos Pré-treinados

Iniciamos o treinamento a partir de pesos já treinados em um dataset gigante (COCO). Isso é chamado de *Transfer Learning* e acelera muito o nosso treinamento.

In [ ]:
# Baixa os pesos convolucionais pré-treinados para o yolov4-tiny
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

### Passo 6: Iniciar o Treinamento!

Este é o comando principal que inicia o processo de treinamento. O Darknet irá carregar seus dados, a configuração da rede, os pesos pré-treinados e começará a aprender.

**Atenção:**
- Este processo pode levar **várias horas**.
- O Colab pode desconectar após um tempo. O progresso é salvo na pasta `backup` a cada 1000 iterações (e o melhor peso é salvo como `_best`).
- Se o treinamento for interrompido, você pode retomá-lo trocando o arquivo de pesos inicial (`yolov4-tiny.conv.29`) pelo último peso salvo na pasta `backup`.

In [ ]:
# Copia seus arquivos de configuração para as pastas corretas dentro do ambiente do Colab
!cp {caminho_base_drive}obj.data ./data/
!cp {caminho_base_drive}obj.names ./data/
!cp {caminho_base_drive}yolov4-tiny-obj.cfg ./cfg/

# Inicia o treinamento
!./darknet detector train data/obj.data cfg/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -dont_show -map

### Passo 7: Testar o Modelo Treinado

Após o treinamento, você pode usar os pesos que foram salvos na sua pasta `backup` no Google Drive para fazer detecções em novas imagens.

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# ATENÇÃO: Modifique os caminhos abaixo conforme necessário
# Caminho para o arquivo de configuração da rede
config_file = './cfg/yolov4-tiny-obj.cfg'
# Caminho para o melhor peso salvo durante o treinamento
weights_file = f'{caminho_base_drive}backup/yolov4-tiny-obj_best.weights'
# Caminho para uma imagem de teste que você queira usar
image_path = '/content/drive/MyDrive/yolo_treinamento/imagem_de_teste.jpg' # COLOQUE UMA IMAGEM DE TESTE AQUI

# Executa o detector na imagem de teste
# O -thresh 0.3 significa que só serão mostradas detecções com confiança acima de 30%
!./darknet detector test data/obj.data {config_file} {weights_file} {image_path} -thresh 0.3

# Mostra a imagem com as detecções (o darknet salva o resultado como 'predictions.jpg')
try:
    img = cv2.imread('predictions.jpg')
    cv2_imshow(img)
except FileNotFoundError:
    print("Não foi possível encontrar o arquivo 'predictions.jpg'. Verifique se o comando de detecção foi executado corretamente.")